# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [163]:
import json

# import the requests library (1 line)
import requests 
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
url = "https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = "/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{url}{status_url}")
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200 : 
    print(req.text)
else : 
    print(req.status_code)

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [164]:
# Set the countries_url variable (1 line)
countries_url = "/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{url}{countries_url}")
# Get the JSON content and store it in the countries variable (1 line)
response = req.json()
# display the request's status code and the countries variable (1 line)
req.json()

{'message': 'The cookie is missing'}

### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [165]:
# Set the cookie_url variable (1 line)
cookie_url = "/cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
cookie = requests.get(f"{url}{cookie_url}").cookies.get_dict()
print(cookie)

{'user_cookie': 'b56847cf-ec84-4d0a-95b9-dfe733b7288d'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [166]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(f"{url}{countries_url}", cookies=cookie).json()
# display the countries variable (1 line)
print(countries)

['ma', 'us', 'ru', 'be', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [167]:
# Set the leaders_url variable (1 line)
leaders_url = "/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(f"{url}{leaders_url}", cookies=cookie)
# display the leaders variable (1 line)
print(leaders.text)

{"message":"Please specify a country"}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [168]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders = requests.get(f"{url}{leaders_url}", cookies=cookie, params = {"country": "ma"})
# display the leaders variable (1 line)
print(leaders.text)

[{"id":"Q57553","first_name":"Mohammed","last_name":"None","birth_date":"1963-08-21","death_date":null,"place_of_birth":"Rabat","wikipedia_url":"https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86","start_mandate":"1999-07-23","end_mandate":null},{"id":"Q69103","first_name":"Hassan","last_name":"None","birth_date":"1929-07-09","death_date":"1999-07-20","place_of_birth":"Rabat","wikipedia_url":"https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF","start_mandate":"1961-02-26","end_mandate":"1999-07-23"},{"id":"Q193874","first_name":"Mohammed","last_name":"None","birth_date":"1909-08-10","death_date":"1961-02-26","place_of_birth":"Fez","wikipedia_url":"https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81","start_mandate":"1957-08-14","end_mandate":

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [169]:
# 4 lines
leaders_per_country = {}
for c in countries : 
    leaders_per_country[c] = requests.get(f"{url}{leaders_url}", cookies=cookie, params = {"country": c}).text

    
print(leaders_per_country)

{'ma': '[{"id":"Q57553","first_name":"Mohammed","last_name":"None","birth_date":"1963-08-21","death_date":null,"place_of_birth":"Rabat","wikipedia_url":"https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86","start_mandate":"1999-07-23","end_mandate":null},{"id":"Q69103","first_name":"Hassan","last_name":"None","birth_date":"1929-07-09","death_date":"1999-07-20","place_of_birth":"Rabat","wikipedia_url":"https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF","start_mandate":"1961-02-26","end_mandate":"1999-07-23"},{"id":"Q193874","first_name":"Mohammed","last_name":"None","birth_date":"1909-08-10","death_date":"1961-02-26","place_of_birth":"Fez","wikipedia_url":"https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81","start_mandate":"1957-08-14","end_m

In [170]:
# or 1 line
leaders_per_country = {c : requests.get(f"{url}{leaders_url}", cookies=cookie, params = {"country": c}).text for c in countries}
print(leaders_per_country)

{'ma': '[{"id":"Q57553","first_name":"Mohammed","last_name":"None","birth_date":"1963-08-21","death_date":null,"place_of_birth":"Rabat","wikipedia_url":"https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86","start_mandate":"1999-07-23","end_mandate":null},{"id":"Q69103","first_name":"Hassan","last_name":"None","birth_date":"1929-07-09","death_date":"1999-07-20","place_of_birth":"Rabat","wikipedia_url":"https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF","start_mandate":"1961-02-26","end_mandate":"1999-07-23"},{"id":"Q193874","first_name":"Mohammed","last_name":"None","birth_date":"1909-08-10","death_date":"1961-02-26","place_of_birth":"Fez","wikipedia_url":"https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81","start_mandate":"1957-08-14","end_m

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [171]:
# < 15 lines
def get_leaders() :
    url = "https://country-leaders.onrender.com"
    leaders_url = "/leaders"
    cookies_url = "/cookie"
    countries_url = "/countries"
    
    cookies = requests.get(f"{url}{cookies_url}").cookies.get_dict()
    countries = requests.get(f"{url}{countries_url}", cookies=cookies).json()
    leaders_per_country = {c: requests.get(f"{url}{leaders_url}", cookies=cookies, params={"country": c}).json() for c in countries}
    return leaders_per_country
    

# Query the enpoint, set the cookies variable and display it (2 lines)
print(get_leaders())

{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [172]:
# 2 lines
leaders_per_country = get_leaders()
leaders_per_country

{'ma': [{'id': 'Q57553',
   'first_name': 'Mohammed',
   'last_name': 'None',
   'birth_date': '1963-08-21',
   'death_date': None,
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86',
   'start_mandate': '1999-07-23',
   'end_mandate': None},
  {'id': 'Q69103',
   'first_name': 'Hassan',
   'last_name': 'None',
   'birth_date': '1929-07-09',
   'death_date': '1999-07-20',
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF',
   'start_mandate': '1961-02-26',
   'end_mandate': '1999-07-23'},
  {'id': 'Q193874',
   'first_name': 'Mohammed',
   'last_name': 'None',
   'birth_date': '1909-08-10',
   'death_date': '1961-02-26',
   'place_of_birth': 'Fez',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [173]:
# 3 lines
wikipedia_url = leaders_per_country['fr'][14]['wikipedia_url']
response = requests.get(wikipedia_url)
print(response.text)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="fr" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Patrice de Mac Mahon — Wikipédia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-fea

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [174]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Patrice de Mac Mahon — Wikipédia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [175]:
# 2 lines
paragraphs = [p.text for p in soup.find_all("p") ]
print(paragraphs)


['Pour les autres membres de la famille, voir Famille de Mac Mahon.\n', 'Pour les articles homonymes, voir MacMahon.\n', '\n', '\n', 'Patrice de Mac Mahon[b], comte de Mac Mahon, 1er duc de Magenta, maréchal de France, né le 13 juin 1808 au château de Sully (Saône-et-Loire) et mort le 17 octobre 1893 à Montcresson (Loiret), est un militaire et homme d’État français, président de la République du 24 mai 1873 au 30 janvier 1879.\n', 'Fait maréchal de France par Napoléon\xa0III, il est sénateur et gouverneur de l’Algérie pendant le Second Empire.\n', 'En 1871, il est à la tête de l’armée régulière (dite «\xa0versaillaise\xa0»), qui réussit à prendre le contrôle de l’ensemble du territoire français en réprimant notamment la garde nationale et les communes insurrectionnelles, dont celle de Paris, refusant de se soumettre au gouvernement Dufaure. Mac Mahon sort victorieux de cette campagne à l’intérieur, qui met fin à la guerre civile.\n', 'Légitimiste, il succède à Adolphe Thiers comme prés

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [176]:
import time
 # <10 lines
# wikipedia_urls = []
# for country, leaders in leaders_per_country.items():
#     for leader in leaders:
#         wikipedia_urls.append(leader['wikipedia_url'])
# 
# first_paragraphs = []
# for idx, url in enumerate(wikipedia_urls):
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content, "html.parser")
#     # Find the first paragraph after the infobox table and the next sibling "p"
#     first_paragraphs.append(soup.find("table", attrs={"class": "infobox"}).find_next_sibling("p").text)
#     print(f"Processed {idx + 1}/{len(wikipedia_urls)}: {url}")
#     time.sleep(0.1) # add a delay to avoid hitting request limits
# 
# # Print the collected first paragraphs
# for i, paragraph in enumerate(first_paragraphs):
#     print(f"URL {i + 1}: {paragraph}")


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [177]:
# 10 lines
# def get_first_paragraph(wikipedia_url : str ) -> str :
#   print(wikipedia_url) # keep this for the rest of the notebook
#   r = requests.get(wikipedia_url)
#   soup = BeautifulSoup(r.content, "html.parser")
#   # Find the first paragraph after the infobox table and the next sibling "p"
#   first_paragraph = soup.find("table", attrs={"class": "infobox"}).find_next_sibling("p").text
#   return first_paragraph

In [178]:
# Test: 3 lines
# wikipedia_url = leaders_per_country['fr'][0]['wikipedia_url']
# test = get_first_paragraph(wikipedia_url)
# print(test)


### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [179]:
# 3 lines
import re 
# s = "François Hollande [fʁɑ̃swa ɔlɑ̃d][n 2] Écouter, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017."
# string = re.sub(r'\s?[\[\(\/;][^)\]/;]*[\)\]/;]|\s?ⓘ|\s?Écouter[?)]|\s?uitspraak|\s?[a-zA-Z-]+ \S+\);', '', s)
# print(string)

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [180]:
# 10 lines
# def get_first_paragraph(wikipedia_url : str ) -> str :
#     print(wikipedia_url) # keep this for the rest of the notebook
#     r = requests.get(wikipedia_url)
#     soup = BeautifulSoup(r.content, "html.parser")
#     untreated = soup.find("table", attrs={"class": "infobox"}).find_next_sibling("p").text
#     first_paragraph = (re.sub(r'\s?[\[\(\/;][^)\]/;]*[\)\]/;]|\s?ⓘ|\s?Écouter[?)]|\s?uitspraak|\s?[a-zA-Z-]+ \S+\);', '', untreated))
#     return first_paragraph


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [188]:
# < 20 lines
# def get_first_paragraph(wikipedia_url : str ) -> str :
#     print(wikipedia_url) # keep this for the rest of the notebook
#     r = requests.get(wikipedia_url)
#     soup = BeautifulSoup(r.content, "html.parser")
#     untreated = soup.find("table", attrs={"class": "infobox"}).find_next_sibling("p").text
#     regex = r'\s?[\[\(\/;][^)\]/;]*[\)\]/;]|\s?ⓘ|\s?Écouter[?)]|\s?uitspraak|\s?[a-zA-Z-]+ \S+\);'
#     if 'fr' in url : regex = r'\s*\[.*?\]|\([^\)]*Écouter\?)|\[\d+\]|\xa0|\n$'
#     elif 'be' in url : regex = r'\[\d+\]|\(uitspraakⓘ\)|uitspraakⓘ\n$'
#     elif 'ru' in url or 'ma' in url : regex = r'\[\d+\]\n$'
#     elif 'en' in url : regex = r'\[\w+\]|\(\/[^\)]+\/[^)]*\)\n$'
#     first_paragraph = (re.sub(regex, '', untreated))
#     return first_paragraph


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [182]:
# < 20 lines
# def get_leaders() :
#     url = "https://country-leaders.onrender.com"
#     leaders_url = "/leaders"
#     cookies_url = "/cookie"
#     countries_url = "/countries"
# 
#     #Fetch cookies and countries data + fetch leaders per country
#     cookies = requests.get(f"{url}{cookies_url}").cookies.get_dict()
#     countries = requests.get(f"{url}{countries_url}", cookies=cookies).json()
#     leaders_per_country = {c: requests.get(f"{url}{leaders_url}", cookies=cookies, params={"country": c}).json() for c in countries}
# 
#     for country, leaders in leaders_per_country.items() : 
#         for leader in leaders : 
#             leader['first_paragraph'] = get_first_paragraph(leader['wikipedia_url'])
# 
#     return leaders_per_country

In [183]:
# Check the output of your function (2 lines)
# print(get_leaders())

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [211]:
# < 25 lines
# def get_leaders() :
#     url = "https://country-leaders.onrender.com"
#     leaders_url = "/leaders"
#     cookies_url = "/cookie"
#     countries_url = "/countries"
# 
#     #Fetch cookies and countries data + fetch leaders per country
#     cookies = requests.get(f"{url}{cookies_url}").cookies.get_dict()
#     countries = requests.get(f"{url}{countries_url}", cookies=cookies).json()
#     leaders_per_country = {c: requests.get(f"{url}{leaders_url}", cookies=cookies, params={"country": c}).json() for c in countries}
#  
#     for country, leaders in leaders_per_country.items() :
#         for leader in leaders :
#             try :
#                 leader['first_paragraph'] = get_first_paragraph(leader['wikipedia_url'])
#             except requests.RequestException as e :
#                 cookies = requests.get(f"{url}{cookies_url}").cookies.get_dict()
#     return leaders_per_country
# get_leaders()

https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D8%A7%D8%A6%D9%85_%D8%A8%D8%A3%D9%85%D8%B1_%D8%A7%D9%84%D9%84%D9%87_%D8%A7%D9%84%D8%B3%D8%B9%D8%AF%D9%8A
https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Ni

{'ma': [{'id': 'Q57553',
   'first_name': 'Mohammed',
   'last_name': 'None',
   'birth_date': '1963-08-21',
   'death_date': None,
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86',
   'start_mandate': '1999-07-23',
   'end_mandate': None,
   'first_paragraph': 'مُحمد السادس بن الحسن الثاني العلوي (مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب من سلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملك الحسن الثاني بعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 يوليو 1999 بالقصر الملكي بالرباط.'},
  {'id': 'Q69103',
   'first_name': 'Hassan',
   'last_name': 'None',
   'birth_date': '1929-07-09',
   'death_date': '1999-07-20',
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%

Check the output of your function again.

In [202]:
# Check the output of your function (1 line)
# get_leaders()

https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D8%A7%D8%A6%D9%85_%D8%A8%D8%A3%D9%85%D8%B1_%D8%A7%D9%84%D9%84%D9%87_%D8%A7%D9%84%D8%B3%D8%B9%D8%AF%D9%8A
https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Ni

error: unbalanced parenthesis at position 28

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [214]:
from requests import Session
def get_first_paragraph(wikipedia_url : str, session : Session) -> str :
    print(wikipedia_url) # keep this for the rest of the notebook
    r = session.get(wikipedia_url)
    soup = BeautifulSoup(r.content, "html.parser")
    untreated = soup.find("table", attrs={"class": "infobox"}).find_next_sibling("p").text
    
    regex = r'\s?[\[\(\/;][^)\]/;]*[\)\]/;]|\s?ⓘ|\s?Écouter|\s?uitspraak|\s?[a-zA-Z-]+ \S+\);'
    if 'fr' in wikipedia_url : regex = r'\s*\[.*?\]|\([^\)]*\\s?Écouter|( Écouter)|(Écouter)|\[\d+\]|\xa0|\n$'
    elif 'nl' in wikipedia_url : regex = r'\[\d+\]|\(uitspraakⓘ\)|uitspraakⓘ|\xa0|\n$'
    elif 'ru' in wikipedia_url or 'ar' in wikipedia_url : regex = r'\[\d+\]|\xa0|\n$'
    elif 'en' in wikipedia_url : regex = r'\[\w+\]|;[^;]*;|\(\/[^\)]+\/[^)]*\)|\xa0|\n$'
    
    first_paragraph = (re.sub(regex, '', untreated))
    return first_paragraph


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [216]:
def print_timing(func):
    '''Create a timing decorator function use @print_timing just above the function you want to time.'''

    def wrapper(*arg):
        start = time.perf_counter()

        # Run the function decorated
        result = func(*arg)

        end = time.perf_counter()
        execution_time = round((end - start), 2)
        print(f'{func.__name__} took {execution_time} sec')
        return result

    return wrapper

In [217]:
# <25 lines
@print_timing
def get_leaders() :
    url = "https://country-leaders.onrender.com"
    leaders_url = "/leaders"
    cookies_url = "/cookie"
    countries_url = "/countries"

    #Fetch cookies and countries data + fetch leaders per country
    cookies = requests.get(f"{url}{cookies_url}").cookies.get_dict()
    countries = requests.get(f"{url}{countries_url}", cookies=cookies).json()
    leaders_per_country = {c: requests.get(f"{url}{leaders_url}", cookies=cookies, params={"country": c}).json() for c in countries}

    session = Session() 
    for country, leaders in leaders_per_country.items() :
        for leader in leaders :
            try :
                leader['first_paragraph'] = get_first_paragraph(leader['wikipedia_url'], session)
            except requests.RequestException as e :
                cookies = requests.get(f"{url}{cookies_url}").cookies.get_dict()
    return leaders_per_country


https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D8%A7%D8%A6%D9%85_%D8%A8%D8%A3%D9%85%D8%B1_%D8%A7%D9%84%D9%84%D9%87_%D8%A7%D9%84%D8%B3%D8%B9%D8%AF%D9%8A
https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Ni

{'ma': [{'id': 'Q57553',
   'first_name': 'Mohammed',
   'last_name': 'None',
   'birth_date': '1963-08-21',
   'death_date': None,
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86',
   'start_mandate': '1999-07-23',
   'end_mandate': None,
   'first_paragraph': 'مُحمد السادس بن الحسن الثاني العلوي (مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب من سلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملك الحسن الثاني بعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 يوليو 1999 بالقصر الملكي بالرباط.'},
  {'id': 'Q69103',
   'first_name': 'Hassan',
   'last_name': 'None',
   'birth_date': '1929-07-09',
   'death_date': '1999-07-20',
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%

Test your new functions.



In [226]:
leaders_per_country = get_leaders()
get_leaders()

https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D8%A7%D8%A6%D9%85_%D8%A8%D8%A3%D9%85%D8%B1_%D8%A7%D9%84%D9%84%D9%87_%D8%A7%D9%84%D8%B3%D8%B9%D8%AF%D9%8A
https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Ni

{'ma': [{'id': 'Q57553',
   'first_name': 'Mohammed',
   'last_name': 'None',
   'birth_date': '1963-08-21',
   'death_date': None,
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86',
   'start_mandate': '1999-07-23',
   'end_mandate': None,
   'first_paragraph': 'مُحمد السادس بن الحسن الثاني العلوي (مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب من سلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملك الحسن الثاني بعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 يوليو 1999 بالقصر الملكي بالرباط.'},
  {'id': 'Q69103',
   'first_name': 'Hassan',
   'last_name': 'None',
   'birth_date': '1929-07-09',
   'death_date': '1999-07-20',
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [227]:
# 3 lines
with open("leaders.json", "w") as file : 
    json.dump(leaders_per_country, file)

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [228]:
# 3 lines

with open("leaders.json", "r") as file :
    json_file = json.load(file)
json_file

{'ma': [{'id': 'Q57553',
   'first_name': 'Mohammed',
   'last_name': 'None',
   'birth_date': '1963-08-21',
   'death_date': None,
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86',
   'start_mandate': '1999-07-23',
   'end_mandate': None,
   'first_paragraph': 'مُحمد السادس بن الحسن الثاني العلوي (مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب من سلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملك الحسن الثاني بعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 يوليو 1999 بالقصر الملكي بالرباط.'},
  {'id': 'Q69103',
   'first_name': 'Hassan',
   'last_name': 'None',
   'birth_date': '1929-07-09',
   'death_date': '1999-07-20',
   'place_of_birth': 'Rabat',
   'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%

Make a function `save(leaders_per_country)` to call this code easily.

In [230]:
# 3 lines
def save(leaders_per_country) :
    with open("leaders.json", "w") as file :
        json.dump(leaders_per_country, file)

In [232]:
# Call the function (1 line)
save(leaders_per_country)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!